In [1]:
import os
import mlflow
import psycopg2 as psycopg
import pandas as pd

In [2]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv('DB_DESTINATION_HOST'),
    "port": os.getenv('DB_DESTINATION_PORT'), 
    "dbname": os.getenv('DB_DESTINATION_NAME'), 
    "user": os.getenv('DB_DESTINATION_USER'), 
    "password": os.getenv('DB_DESTINATION_PASSWORD')
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [3]:
df.head()

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,17,8191-XWSZG,2015-10-01,NaT,One year,No,Mailed check,20.65,1022.95,None,...,None,None,None,None,Female,0,No,No,No,0
1,59,3957-SQXML,2017-04-01,NaT,Two year,No,Credit card (automatic),24.95,894.30,None,...,None,None,None,None,Female,0,Yes,Yes,Yes,0
2,148,6837-BJYDQ,2019-11-01,NaT,One year,No,Mailed check,19.60,61.35,None,...,None,None,None,None,Male,0,No,No,No,0
3,482,0486-LGCCH,2019-03-01,NaT,Two year,No,Mailed check,19.65,225.75,None,...,None,None,None,None,Male,0,Yes,Yes,No,0
4,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,None,0


In [4]:
# Получаем названия колонок из датафрейма
columns = df.columns.tolist()

In [5]:
columns

['id',
 'customer_id',
 'begin_date',
 'end_date',
 'type',
 'paperless_billing',
 'payment_method',
 'monthly_charges',
 'total_charges',
 'internet_service',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'gender',
 'senior_citizen',
 'partner',
 'dependents',
 'multiple_lines',
 'target']

In [6]:
# Создаем строку с названиями колонок, разделенными запятыми
columns_string = ','.join(columns)

In [7]:
columns_string

'id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target'

In [8]:
# Записываем строку в файл
with open('columns.txt', 'w') as f:
    f.write(columns_string)

In [9]:
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

for col in counts_columns:
    column_stat = df[col].value_counts().to_dict()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}
    stats.update(column_stat)

stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max()
stats["monthly_charges_mean"] = df["monthly_charges"].mean()
stats["monthly_charges_median"] = df["monthly_charges"].median()
stats["total_charges_min"] = df["total_charges"].min()
stats["total_charges_max"] = df["total_charges"].max()
stats["total_charges_mean"] = df["total_charges"].mean()
stats["total_charges_median"] = df["total_charges"].median()
stats["unique_customers_number"] = df["customer_id"].nunique()
stats["end_date_nan"] = df["end_date"].isna().sum()

print(stats)

{'type_Month-to-month': 3875, 'type_Two year': 1695, 'type_One year': 1473, 'paperless_billing_Yes': 4171, 'paperless_billing_No': 2872, 'internet_service_Fiber optic': 3096, 'internet_service_DSL': 2421, 'online_security_No': 3498, 'online_security_Yes': 2019, 'online_backup_No': 3088, 'online_backup_Yes': 2429, 'device_protection_No': 3095, 'device_protection_Yes': 2422, 'tech_support_No': 3473, 'tech_support_Yes': 2044, 'streaming_tv_No': 2810, 'streaming_tv_Yes': 2707, 'streaming_movies_No': 2785, 'streaming_movies_Yes': 2732, 'gender_Male': 3555, 'gender_Female': 3488, 'senior_citizen_0': 5901, 'senior_citizen_1': 1142, 'partner_No': 3641, 'partner_Yes': 3402, 'dependents_No': 4933, 'dependents_Yes': 2110, 'multiple_lines_No': 3390, 'multiple_lines_Yes': 2971, 'target_0': 5174, 'target_1': 1869, 'data_length': 7043, 'monthly_charges_min': 18.25, 'monthly_charges_max': 118.75, 'monthly_charges_mean': 64.76169246059918, 'monthly_charges_median': 70.35, 'total_charges_min': 18.8, 'to

In [10]:
df.to_csv("users_churn.csv", index=False)

In [11]:
# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env


# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

YOUR_NAME = "ivan_panchenko" # введите своё имя для создания уникального эксперимента
assert YOUR_NAME, "введите своё имя в переменной YOUR_NAME для создания уникального эксперимента"

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = f"churn_{YOUR_NAME}"
RUN_NAME = "data_check"

# тестовые данные
METRIC_NAME = "test_metric"
METRIC_VALUE = 0

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")




In [12]:

# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) # ваш код здесь

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id # ваш код здесь
    
    # логируем метрики эксперимента
    # предполагается, что переменная stats содержит словарь с метриками,
    # объявлять переменную stats не надо,
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(stats) # ваш код здесь
    
    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    mlflow.log_artifact('columns.txt', artifact_path='dataframe') # ваш код здесь
    mlflow.log_artifact('users_churn.csv', artifact_path='dataframe') # ваш код здесь


experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id) # ваш код здесь


# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert run.info.status == 'FINISHED' # ваш код здесь

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
os.remove('columns.txt')
os.remove('users_churn.csv')

RestException: RESOURCE_ALREADY_EXISTS: Experiment(name=churn_ivan_panchenko) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "experiments_name_key"
DETAIL:  Key (name)=(churn_ivan_panchenko) already exists.

[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (%(name)s, %(artifact_location)s, %(lifecycle_stage)s, %(creation_time)s, %(last_update_time)s) RETURNING experiments.experiment_id]
[parameters: {'name': 'churn_ivan_panchenko', 'artifact_location': '', 'lifecycle_stage': 'active', 'creation_time': 1754226105833, 'last_update_time': 1754226105833}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [23]:
import mlflow
import mlflow.catboost
from catboost import CatBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate a simple dataset
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a simple CatBoost model
model = CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, verbose=0)
model.fit(X_train, y_train)

# Set up MLflow
mlflow.set_experiment("CatBoost_Example")

# Log the model
experiment_id = '13'

with mlflow.start_run(run_name='log_model2', experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    # Log model parameters
    mlflow.log_params(model.get_params())
    
    # Log model performance
    train_accuracy = model.score(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("test_accuracy", test_accuracy)
    
    # Log the model
    model_info = mlflow.catboost.log_model(cb_model=model, artifact_path="models")

print(f"Model logged with run_id: {run_id}")
print(f"Model URI: {model_info.model_uri}")

# Now you can load the model
loaded_model = mlflow.catboost.load_model(model_uri=model_info.model_uri)
print("Model loaded successfully")

Model logged with run_id: 6d010cff2aa54df5a216dd5fa0e3c48e
Model URI: runs:/6d010cff2aa54df5a216dd5fa0e3c48e/models


Model loaded successfully


Mock model logged with run_id: 7f4d1b2d8973422faf778cc2fc95203e
Model URI: runs:/7f4d1b2d8973422faf778cc2fc95203e/models


In [21]:
loaded_model = mlflow.catboost.load_model(model_uri=model_info.model_uri)

In [24]:
class CatboostModelProba(mlflow.pyfunc.PythonModel):

    def __init__(self, model):
        super().__init__()
        self._model = model
    
    def predict(self, context, model_input):
        probas = self._model.predict_proba(model_input)[:, 1]
        
        return probas

custom_model = CatboostModelProba(model)

In [25]:
import mlflow.pyfunc
import numpy as np

class CatboostModelProba(mlflow.pyfunc.PythonModel):

    def __init__(self, model):
        self.model = model
    
    def predict(self, context, model_input):
        base_predictions = self.model.predict(model_input)
        predictions = np.sqrt(np.abs(base_predictions))

        return predictions

In [27]:
experiment_id = '0'
custom_model = CatboostModelProba(model) # ваш код инициализации модели через CustomMlflowModel


with mlflow.start_run(
    run_name="custom_model",
    experiment_id=experiment_id,
) as run:
    
    model_info = mlflow.pyfunc.log_model(artifact_path="models_custom_model", python_model=custom_model) # напишите код логирования модели тут

In [28]:
loaded_custom_model = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)